In [1]:
import numpy as np
import gurobipy as gp

# Question 1
This is just a practice problem if you want to learn how branch and bound works. If you don't then you don't need to do this question, it is worth 0 points on the assignment!

I did not do this question.

# Question 2

A company is thinking about building new facilities in Austin and Dallas. Here is the relevant data.
    
                        Capital Needed          Expected Profit
    Factory in Austin:     $ 6 M                    $ 9 M

    Factory in Dallas:     $ 3 M                    $ 5 M

    Warehouse in Austin:   $ 5 M                    $ 6 M

    Warehouse in Dallas:   $ 2 M                    $ 4 M
    
Total capital available for investment is $11M. You can’t build more than one factory (warehouse) in one place. At most one warehouse must be built in Austin or Dallas. At least one factory must be built in Austin or Dallas. Find the optimal investment strategy.

In [54]:
obj = np.array([9,5,6,4])
sense = ['<','>','<']
A = np.zeros((3,4))
A[0,:] = [6,3,5,2]
A[1,:] = [1,1,0,0]
A[2,:] = [0,0,1,1]
b = np.array([11,1,1])

In [64]:
avdMod = gp.Model()
avdMod_x = avdMod.addMVar(len(obj),vtype=['B']*len(obj))
avdMod_con = avdMod.addMConstrs(A, avdMod_x, sense, b)
avdMod.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

avdMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
avdMod.optimize()

Is building the Factory in Dallas part of the optimal investment strategy?

The array is read as: 

( Factory in Austin, Factory in Dallas, Warehouse in Austin, Warehouse in Dallas)

In [56]:
avdMod_x.x

array([1., 1., 0., 1.])

### Answer:
Yes, building a factory in Dallas is in the optimal investment strategy.

# Question 3

Western Airlines wants to design a hub system in the United States. Each hub is used for connecting flights to and from cities within 1000 miles of the hub. Western runs flights among the following cities: Atlanta (ATL), Boston (BOS), Chicago (CHI), Denver (DEN), Houston (HOU), Los Angeles (LAX), New Orleans (NO), New York (NY), Pittsburgh (PIT), Salt Lake City (SLC), San Francisco (SF), and Seattle (SEA). The company wants to determine the smallest number of hubs it needs to cover all these cities, where a city is covered if it is within 1000 miles of at least one hub. The table below lists which cities are within 1000 miles of other cities. For example, if a hub was placed at Boston (BOS), it could cover the cities of Boston, New York, and Pittsburgh.

            ATL:   BOS:   CHI:   DEN:   HOU:   LAX:    NO:   NY:   PIT:   SCL:    SF:   SEA:
    ATL:     X      0      X      0      X      0      X     X      X      0      0      0
    BOS:     0      X      0      0      0      0      0     X      X      0      0      0
    CHI:     X      0      X      0      0      0      X     X      X      0      0      0
    DEN:     0      0      0      X      0      0      0     0      0      X      0      0 
    HOU:     X      0      0      0      X      0      X     0      0      0      0      0
    LAX:     0      0      0      0      0      X      0     0      0      X      X      0
    NO:      X      0      X      0      X      0      X     0      0      0      0      0
    NY:      X      X      X      0      0      0      0     X      X      0      0      0
    PIT:     X      X      X      0      0      0      0     X      X      0      0      0
    SCL:     0      0      0      X      0      X      0     0      0      X      X      X
    SF:      0      0      0      0      0      X      0     0      0      X      X      X
    SEA:     0      0      0      0      0      0      0     0      0      X      X      X
    
Formulate and solve this problem as a binary integer program.

In [59]:
obj = np.array([1,1,1,1,1,1,1,1,1,1,1,1])
sense = ['>']* 12
A = np.zeros((12,12))
A[0,:] = [1,0,1,0,1,0,1,1,1,0,0,0] # ATL
A[1,:] = [0,1,0,0,0,0,0,1,1,0,0,0] # BOS
A[2,:] = [1,0,1,0,0,0,1,1,1,0,0,0] # CHI
A[3,:] = [0,0,0,1,0,0,0,0,0,1,0,0] # DEN

A[4,:] = [1,0,0,0,1,0,1,0,0,0,0,0] # HOU
A[5,:] = [0,0,0,0,0,1,0,0,0,1,1,0] # LAX
A[6,:] = [1,0,1,0,1,0,1,0,0,0,0,0] # NO
A[7,:] = [1,1,1,0,0,0,0,1,1,0,0,0] # NY

A[8,:] = [1,1,1,0,0,0,0,1,1,0,0,0] # PIT
A[9,:] = [0,0,0,1,0,1,0,0,0,1,1,1] # SLC
A[10,:] = [0,0,0,0,0,1,0,0,0,1,1,1] # SF
A[11,:] = [0,0,0,0,0,0,0,0,0,1,1,1] # SEA
b = np.array([1]*12)

In [65]:
wstMod = gp.Model()
wstMod_x = wstMod.addMVar(len(obj),vtype=['B']*len(obj)) 
wstMod_con = wstMod.addMConstrs(A, wstMod_x, sense, b)
wstMod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

wstMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
wstMod.optimize()

In [61]:
wstMod_x.x

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.])

Is SLC a hub in the optimal solution?

In [58]:
wstMod_x.x[9]

1.0

### Answer: 
Yes, SLC is a hub in the optimal solution.

# Question 4

A paper mill cuts the rolls of paper into different widths to satisfy customers’ demand. In this problem, assume the original rolls of paper are 120 inches wide. The table below shows the orders received by the paper mill.

    WIDTH    QUANTITY
     25       233
     37       148
     54       106

A 120 inch roll can be cut in many ways. For example, we can cut four 25-inch rolls while wasting the
remaining 20 inches; we can also cut one 25-inch, one 37-inch, and one 54-inch. In the second case, only 4 inches is wasted.

Develop and solve an integer program to minimize the waste from cutting to satisfy all orders. For more information, you can check out the cutting stock problem on wikipedia: https://en.wikipedia.org /wiki/Cutting_stock_problem     (https://en.wikipedia.org/wiki/Cutting_stock_problem)


In [2]:
obj = np.array([90,70,45,20,82,46,9,66,12,58,4,21,33,8,29,16,41])
sense = ['='] * 3
                       
A = np.zeros((3,17))
A[0,:] = [1,2,3,4,0,0,0,0,0,1,1,1,2,3,0,2,1] # 25 in width
A[1,:] = [0,0,0,0,1,2,3,0,0,1,1,2,1,1,1,0,0] # 37 in width
A[2,:] = [0,0,0,0,0,0,0,1,2,0,1,0,0,0,1,1,1] # 54 in width

b = np.array([233,148,106]) # How much is needed

In [3]:
paperMod = gp.Model()
paperMod_x = paperMod.addMVar(len(obj),vtype=['I']*len(obj)) # 'C' or 'I' or 'B'
paperMod_con = paperMod.addMConstrs(A, paperMod_x, sense, b)
paperMod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

paperMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
paperMod.optimize()

Restricted license - for non-production use only - expires 2022-01-13


How many rolls get cut into the pattern such that there are 3 cuts of 25 inches, and 1 cut of 37 inches (this pattern results in 8" waste)?

In [4]:
paperMod_x.x

array([  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 106.,
         0.,   0.,  42.,   0.,   0.,   0.])

### Answer: 
42 rolls should be cut into the pattern of 3 cuts of 25 inches, and 1 cut of 37 inches.

# Question 5

The days-off scheduling problem must be solved routinely by businesses that operate 6 or 7 days a week. Examples include hospitals, airlines, municipal transportation companies, and the postal service. The most common example is the (5,7)-cyclic staffing problem. The objective of it is to minimize the cost of assigning workers to a 7-day cyclic schedule so that
1) Sufficient workers are available every day.

2) Each person works 5 consecutive days and is idle to the remaining 2 days.

Here is the table showing the cost of having an employee for each day and the number of employees required for each day.

                  SUN.  MON.  TUES.  WED.  THUR.  FRI.  SAT.
       COST       90    60    60     60    60     60    90
       REQUIRED   5     13    12     10    14     8     6
 
For example, the pattern that one works from Sunday to Thursday costs 90 + 60 × 4 = 330. 

Formulate and solve an integer programming problem to represent this problem.

In [7]:
obj = np.array([330,300,330,360,360,360,360])
sense = ['>']*7
A = np.zeros((7,7))
A[0,:] = [1,0,0,1,1,1,1] # Sunday off
A[1,:] = [1,1,0,0,1,1,1] # Monday off
A[2,:] = [1,1,1,0,0,1,1] # Tuesday off
A[3,:] = [1,1,1,1,0,0,1] # Wednesday off
A[4,:] = [1,1,1,1,1,0,0] # Thursday off
A[5,:] = [0,1,1,1,1,1,0] # Friday off
A[6,:] = [0,0,1,1,1,1,1] # Saturday off

b = np.array([5,13,12,10,14,8,6])

In [8]:
daysoffMod = gp.Model()
daysoffMod_x = daysoffMod.addMVar(len(obj),vtype=['I']*len(obj)) # 'C' or 'I' or 'B'
daysoffMod_con = daysoffMod.addMConstrs(A, daysoffMod_x, sense, b)
daysoffMod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

daysoffMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
daysoffMod.optimize()

How many employees work Monday-Friday?

In [9]:
daysoffMod_x.x

array([ 1.,  8.,  2., -0.,  3.,  1., -0.])

### Answer: 
8 employees work Monday - Friday.